<a href="https://colab.research.google.com/github/YanaYanovna/Colab/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%5Bhomework_4_2_%5Dgame_of_thrones_prediction_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
!gdown 1h99toeF7lZ2I3iJwehgKO-QQmDaOe_O3 # test dataset
!gdown 1XL0VTygpZj-ZAuTNRBgApZTPQyNDnT-v # train dataset
!gdown 1M14conWjAW2QLoyCXbHEAy8bql2f99eF

Downloading...
From: https://drive.google.com/uc?id=1h99toeF7lZ2I3iJwehgKO-QQmDaOe_O3
To: /content/game_of_thrones_test.csv
100% 37.3k/37.3k [00:00<00:00, 42.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XL0VTygpZj-ZAuTNRBgApZTPQyNDnT-v
To: /content/game_of_thrones_train.csv
100% 138k/138k [00:00<00:00, 23.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1M14conWjAW2QLoyCXbHEAy8bql2f99eF
To: /content/submission.csv
100% 2.74k/2.74k [00:00<00:00, 7.61MB/s]


In [ ]:
data = pd.read_csv('/content/game_of_thrones_train.csv', index_col='S.No')

In [ ]:
# Подсказка
cult = {
    'Summer Islands': ['summer islands', 'summer islander', 'summer isles'],
    'Ghiscari': ['ghiscari', 'ghiscaricari',  'ghis'],
    'Asshai': ["asshai'i", 'asshai'],
    'Lysene': ['lysene', 'lyseni'],
    'Andal': ['andal', 'andals'],
    'Braavosi': ['braavosi', 'braavos'],
    'Dornish': ['dornishmen', 'dorne', 'dornish'],
    'Myrish': ['myr', 'myrish', 'myrmen'],
    'Westermen': ['westermen', 'westerman', 'westerlands'],
    'Westerosi': ['westeros', 'westerosi'],
    'Stormlander': ['stormlands', 'stormlander'],
    'Norvoshi': ['norvos', 'norvoshi'],
    'Northmen': ['the north', 'northmen'],
    'Free Folk': ['wildling', 'first men', 'free folk'],
    'Qartheen': ['qartheen', 'qarth'],
    'Reach': ['the reach', 'reach', 'reachmen'],
}

def simplify_culture(culture):
    for key, values in cult.items():
        if str(culture).lower() in values:
            return key
    return 'unknown'

median_value = data['age'].median()
columns_to_fill = ['isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse']

In [ ]:
data['isPopular'] = data['popularity'].apply(lambda x: 1 if x > 0.5 else 0)
data['boolDeadRelations'] = data['numDeadRelations'].apply(lambda x: 1 if x > 0 else 0)
data['culture'] = data['culture'].apply(simplify_culture)
data['age'] = data['age'].fillna(median_value)
data[columns_to_fill] = data[columns_to_fill].fillna(0)
data.drop(columns=['name', 'title', 'dateOfBirth', 'mother', 'father', 'heir', 'house', 'spouse', 'popularity', 'numDeadRelations'], inplace=True)

In [ ]:
encoder = OneHotEncoder(sparse_output=False, dtype=float, handle_unknown='ignore')
one_hot_encoded = encoder.fit_transform(data[['culture']])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(), index=data.index)

In [ ]:
data = pd.concat([data, one_hot_df], axis=1).drop('culture', axis=1)

In [ ]:
X = data.drop(columns=['isAlive']).values
y = data['isAlive'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

**Задание 2.1.** Импортируйте следующие модели из библиотеки `sklearn`
* LogisticRegression
* RandomForestClassifier
* AdaBoostClassifier
* GaussianProcessClassifier
* GaussianNB
* KNeighborsClassifier
* SVC
* DecisionTreeClassifier


In [ ]:
model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)
print("Accuracy на тренировочном наборе: %.4f" % accuracy_score(y_train, model.predict(X_train_scaled)))
print("Accuracy на тестовом наборе: %.4f" % accuracy_score(y_test, model.predict(X_test_scaled)))

Accuracy на тренировочном наборе: 0.8867
Accuracy на тестовом наборе: 0.8013


In [ ]:
model = RandomForestClassifier()

param_grid = {
    'max_depth': np.arange(1, 5),
    'min_samples_leaf': np.arange(1, 4),
    'n_estimators': [100, 1000]
}

gridsearch = GridSearchCV(model, param_grid, refit=True, scoring=make_scorer(accuracy_score))
gridsearch.fit(X_train_scaled, y_train)

print(gridsearch.best_params_)

best_model = gridsearch.best_estimator_

print("Accuracy на тренировочном наборе: %.4f" % accuracy_score(y_train, best_model.predict(X_train_scaled)))
print("Accuracy на тестовом наборе: %.4f" % accuracy_score(y_test, best_model.predict(X_test_scaled)))

{'max_depth': 4, 'min_samples_leaf': 2, 'n_estimators': 100}
Accuracy на тренировочном наборе: 0.7960
Accuracy на тестовом наборе: 0.8237


In [ ]:
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
model = RandomForestClassifier(max_depth=4, min_samples_leaf=2, n_estimators=100)
model.fit(X_scaled, y)
print("Accuracy на тренировочном наборе: %.4f" % accuracy_score(y, model.predict(X_scaled)))

Accuracy на тренировочном наборе: 0.8048


In [ ]:
dat = pd.read_csv('/content/game_of_thrones_test.csv', index_col='S.No')

In [ ]:
dat['isPopular'] = dat['popularity'].apply(lambda x: 1 if x > 0.5 else 0)
dat['boolDeadRelations'] = dat['numDeadRelations'].apply(lambda x: 1 if x > 0 else 0)
dat['culture'] = dat['culture'].apply(simplify_culture)
dat['age'] = dat['age'].fillna(median_value)
dat[columns_to_fill] = dat[columns_to_fill].fillna(0)
dat.drop(columns=['name', 'title', 'dateOfBirth', 'mother', 'father', 'heir', 'house', 'spouse', 'popularity', 'numDeadRelations'], inplace=True)

In [ ]:
one_hot_encode = encoder.fit_transform(dat[['culture']])
one_hot_d = pd.DataFrame(one_hot_encode, columns=encoder.get_feature_names_out(), index=dat.index)

In [ ]:
dat = pd.concat([dat, one_hot_d], axis=1).drop('culture', axis=1)

In [ ]:
Xx = dat.values

In [ ]:
Xx_scaled = scaler.transform(Xx)

In [ ]:
pre = model.predict(Xx_scaled)

In [ ]:
submission = pd.DataFrame(pre, index=dat.index, columns=['isAlive'])

In [ ]:
# submission = pd.read_csv("/content/submission.csv", index_col='S.No')

In [ ]:
submission.to_csv("/content/new_submission.csv", index=False)